# FTT model including constraint and learning

In this notebook the cell magic %latexmodelgrab is used. It allows the specification of segments of the model. 

In [ ]:
#This is code to manage dependencies if the notebook is executed in the google colab cloud service
if 'google.colab' in str(get_ipython()):
  import os
  os.system('apt -qqq install graphviz')
  os.system('pip -qqq install ModelFlowIb ipysheet  --no-dependencies ')

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from IPython.core.display import HTML,Markdown,Latex


from modelclass import model
import modeljupytermagic

# some useful stuf
model.widescreen()
pd.set_option('display.max_rows', None, 'display.max_columns', 10, 'display.precision', 4)

## Define the segments of the model
By defining the a model as segments the whold model does not have to be defines in one cell. This simplifies the specification. 
And debugging.

As many segments will use the lists of technologies, segments which name starts 
with list will be appended to each segment. 

## Changes

- for each sector change demand to production
- calculate average cost for power generation 
- change the cost equations so the cost include learning 

### First the technologies are defined

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=list1

Two lists of technology  are defined: 
    
$List \; i =\{Oil, Coal, Gas, Biomass, Solar, Wind, Hydro, Geothermal\} \\  
         fosile: \{  1 , 1, 1, 0, 0, 0, 0, 0 \}$  

$List \; j = \{Oil, Coal, Gas, Biomass, Solar, Wind, Hydro, Geothermal \}$

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=text1

Any number of technology can be specified (limited by the avaiable memory)

The time index $_t$ is implicit. 

### Preferences

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=preferences 

Each technology is compared to all other based on the percieved costs and the preferences (choice likelihood) $F^{i,j}$ are calculated.
    
For all technologies $F^{i,j}+F^{j,i} = 1 $

\begin{equation}
\label{eq:preferences}
\underbrace{F^{i,j}}_{Preferences} = \frac{1}{
1+exp(
    \frac{(Cost\_argumented^{i}-Cost\_argumented^{j})}{\sigma^{i,j}} )}
\end{equation}



### Share dynamic 

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=Share 
\begin{equation}
\label{eq:SHARES2}
\Delta Share^{i} = \sum_{j}(Share^{i} \times Share^{j} \times
                            (\frac{F^{i,j}}{\tau^{j}}
                           - \frac{F^{j,i}}{\tau^{i}}))
\end{equation}

### Sigma

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=sigma 
The percieved width in the choice function

\begin{equation}
\label{eq:sigma}
\sigma^{i,j} = \sqrt{(\sigma^{i})^{(2)} \cdot (\sigma^{j})^{(2)}}
\end{equation}

### Total Demand 

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=demand 

\begin{equation}
\label{eq:UD_total}
UsefulDemand\_total  = UsefulDemand\_total_{t-1} \cdot 
(1+\frac{UsefulDemand\_total\_growth}{100})
\end{equation}

### Production  

In [ ]:
%%latexmodelgrab mftt  display segment=production 

The production from each sector can be calculated. The same goes fdor 

\begin{equation}
\label{eq:production}
Production^{i} = Share^{i} \cdot UsefulDemand\_total 
\end{equation}

\begin{equation}
\label{eq:acc_production}
Production\_accumulated^{i} = Production\_accumulated^{i}_{t-1} + Production^{i}
\end{equation}

### price learning 

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=endogeneous_prices 
1  FTT model including constraint and learning
1.1  Define th

The price of new technology falls in line with its accumulated use through a number of effects.

In princip it depends in global accumulated production. But lacking the data, we just use the local accumulated production. There are many
possible functional forms. This is just one example. 

\begin{equation}
\label{eq:change_production}
Change\_accumulated\_Production^{i} = \frac{Production\_accumulated^{i}}{Production\_accumulated^{i}_{t-1}}
\end{equation}

\begin{equation}
\label{eq:price_impact}
Cost^{i} = Cost^{i}_{t-1} \cdot (Change\_accumulated\_Production^{i})^{(-cost\_elasticity^{i})} 
\end{equation}

### Imposing min and max shares  

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=min_max 

Sometime there can be constarint (min and/or max) on the shares (or the demand). To enforce constraints
 - a penalty is added to the percieved cost if the share violate the max constrain.  
 - a deduction is subtracted from the cost if the shares violates the min constrain. .  



We really want to have this equations: 
\begin{equation}
\label{eqno:maxout}
\underbrace{penalty\_max^{i}}_{Penalty\_max} =  20 * (Share^{i} > Share\_max^{i}) 
\end{equation}

However in order to get more numerical stability an equation which smooth out the logical expression: $Share^{i} > Share\_max^{i}$  is used: 

\begin{equation}
\label{eq:maxoutlogit}
\underbrace{penalty\_max\_l^{i}}_{Penalty\_max} =  1000 * logit^{-1}(1000*(Share^{i} - Share\_max^{i})) 
\end{equation}



We want this
\begin{equation}
\label{eqno:minout}
deduction\_min^{i} = Cost^{i}* 0.99 * (Share^{i} < Share\_min^{i})
\end{equation}

And implement it like this

\begin{equation}
\label{eq:minoutlogit}
deduction\_min\_l^{i} = Cost^{i}* 0.99 * logit^{-1}(100*(Share\_min^{i}-Share^{i})  )
\end{equation}




which is the value which enters the equation for prefereences ($F^{i,j}$)



### Argumented cost 

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=Argumented_cost 

\begin{equation}
\label{eq:Z_costbarrier}
Cost\_argumented^{i} = Cost^{i} +  penalty\_max\_l^{i} -  deduction\_min\_l^{i}
\end{equation}

### Average cost 

In [ ]:
%%latexmodelgrab mftt  display segment=average_cost 

To feed into the main model the average cost is calculated. 
This is done on the actual cost, not the argumented cost which can deviate from
the actual cost in order to impose constraints. 

\begin{equation}
\label{eq:Average_cost}
Cost\_average = \sum_{i}(Cost^{i} \cdot Share^{i} ) 
\end{equation}


### Max cost 

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=average_cost 

To feed into the main model the max cost is calculated. 
This is done on the actual cost, not the argumented cost which can deviate from
the actual cost in order to impose constraints. 


\begin{equation}
\label{eq:max_cost}
max\_total  = \max_{i}(Share^{i}) 
\end{equation}

### Check, that the shares add up to one. 

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=check 

\begin{equation}
\label{eq:check_shares}
Share\_total  = \sum_{i}(Share^{i})
\end{equation}

### Create the share for fosile fuels 

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=fosile 

\begin{equation}
\label{eq:share_fosile}
Share\_\_fosile  = \sum_{i=fosile}(Share^{i})
\end{equation}

### Create equations just for fosile fuels

In [ ]:
%%latexmodelgrab mftt  ndisplay segment=fosilesharetest 

\begin{equation}
\label{eq:SHARES3}
\forall [i=fosile]\:\:  fosileshare^{i} = Share^{i} 
\end{equation}



## Combine all the segments

In [ ]:
%%latexmodelgrab mftt  ndisplay render=False
# Combined model

In [ ]:
display(Markdown(mftt.equations_latex))

### All phases of the combined model can be displayed:

In [ ]:
if 0: 
    mftt_latex_model_instance.show

## Create some input data and start values 

In [ ]:
%%dataframe  startvalues  show  periods=2  melt t
                share_  Production_accumulated_
Oil           0.0828         100
Coal          0.0000         100
Gas           0.0000         100 
Biomass       0.0101         100
Solar         0.0085         100
Wind          0.1276         100
Hydro         0.3381         100
Geothermal    0.4329         100

In [ ]:
%%dataframe  exoinit  nshow  periods=100 
USEFULDEMAND_TOTAL  USEFULDEMAND_TOTAL_GROWTH 
     100                    2

In [ ]:
%%dataframe  exoparam   nshow  periods=100 melt
                Oil     Coal    Gas      Biomass     Solar   Wind   Hydro     Geothermal
Cost_          83.5     83.5   70.1        134.0     96.9    81.8   108.0       102.5
Sigma_         28.1     28.1   25.0        106.6     48.4    46.1    77.1        36.0
Tau_           40.0     40.0   30.0         40.0     25.0    25.0    80.0        40.0
share_max_      200       200   200         200       200     200     200         200
share_min_      -10      -100  -100        -100      -100    -100    -100         -100

In [ ]:
baseline = pd.concat([startvalues_melted,exoparam_melted,exoinit],axis=1).fillna(0).pipe(lambda df:df.loc[:,sorted(df.columns)])
baseline.head().T

## Some housekeeping 

In [ ]:
sigma_names = ' '.join(name for name in mftt.exogene if name.startswith('SIGMA'))
tau_names = ' '.join(name for name in mftt.exogene if name.startswith('TAU'))

print(sigma_names)
print(tau_names)

In [ ]:
mftt.group_dict = {
    'share__[!M]* share_total' : 'Shares',
    'c_t* cost*'   :  'Cost' ,
    'useful*':  'Demand',
    'pro*':  'Production',
    'deduct*':  'cost deduction for minimum ',
    'penal*':  'cost penalty  for maximum ',
    '*'      :  'all'
}

## Make a baseline solution 

In [ ]:
mftt.keep_solutions = {}   # to clean up with new experiments
res = mftt(baseline,silent=1,first_test=10,init=True,keep='Baseline',alfa=0.1,ljit=0)

## Create some senarios

In [ ]:
mftt.keep_solutions = {}   # to clean up with new experiments
    
lookat = 'hydro'
for change_cost in [0.0 , -10.0, -25.0, -50.0 , 10.0 , 25.0, 50.0]:
    alternative = baseline.upd(f'''cost__{lookat} %  {change_cost}
                                   share_max__hydro = 0.4 
                                   {sigma_names} = 10
                                   {tau_names}  = 5 
                                   ''')
    _ = mftt(alternative,keep = f'Cost of {lookat} up {change_cost}%',
             alfa=0.1,max_iterations=300,silent=1)

## Display the results 

In [ ]:
mftt.keep_show(select_scenario=1)

## Look at the plots of shares

In [ ]:
sharenames = [name for name in mftt.endogene if name.startswith('SHARE__') and not ('_M' in name or '_FOSILE' in name or '_TOTAL' in name)]

In [ ]:
mftt[sharenames].df.plot.area();

## Waterfall plots of share changes

In [ ]:
share_out = (100*pd.concat([ pd.DataFrame(df.loc['2120',sharenames] -df.loc['2021',sharenames],columns= [scenario]) 
                       for scenario,df in mftt.keep_solutions.items()]
                      ,axis=1).
                       pipe(lambda df: df.rename({i:i.split('__')[1] for i in df.index}))) 

display(Markdown(share_out.to_html()))

In [ ]:
from modelvis import waterplot
fig = waterplot(share_out,title='Change in share from 2020 to 2120',threshold=0.001);
#fig.set_constrained_layout(True)

In [ ]:
mftt['production_ac*']   

In [ ]:
2+2